In [1]:
from collections import defaultdict
from pprint import pprint
import time
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from balance_date import BalanceDate

import utils
pd.options.display.max_rows = 10 
%matplotlib inline

from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.model_selection import train_test_split
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=0)  
from sklearn.externals import joblib  
from sklearn.feature_selection import RFE,RFECV

from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb
import time


from tqdm import tqdm, trange
with open("./config.json") as f:
    config = json.loads(f.read())

配置文件
{'BTK': './out/BTK.faste',
 'aaindex': './data/aaindexmatrix.txt',
 'all_feature': './data/all_feature.csv',
 'feature_kind': './data/features_kind.xls',
 'feature_selected': {'LightGBM_rfe_100': './out/20210203/10cv_3class_feature_select_LightGBM_direct_100.rfe',
                      'LightGBM_rfe_20': './out/20210203/10cv_3class_feature_select_LightGBM_direct_20.rfe',
                      'LightGBM_rfe_30': './out/20210108/10cv_3class_feature_select_LightGBM_30.rfe',
                      'LightGBM_rfe_50': './out/20210203/10cv_3class_feature_select_LightGBM_direct_50.rfe',
                      'RandomForeast': './out/20201224/10cv_3class_feature_select_RandomForeast.rfecv',
                      'RandomForeast_rfe_30': './out/20201226/10cv_3class_feature_select_RandomForeast_30.rfe',
                      'XGBoost': './out/20201224/10cv_3class_feature_select_XGBoost.rfecv',
                      'XGBoost_rfe_30': './out/20201226/10cv_3class_feature_select_XGBoost_30.rfe',
  

d:\application\annaconda\envs\ponsol_web\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
FEATURE = BalanceDate.get_feature() 
OUT_PATH = os.path.join(".", "out", time.strftime("%Y%m%d")) 
if not os.path.exists(OUT_PATH):
    os.makedirs(OUT_PATH)
print("feature：", FEATURE)
print("out path：", OUT_PATH)

feature： 0             sift
1             hits
2       ANDN920101
3       ARGP820101
4       ARGP820102
           ...    
1076    NonPolarAA
1077       PolarAA
1078     ChargedAA
1079         PosAA
1080         NegAA
Length: 1081, dtype: object
out path： .\out\20210525


# read data

In [3]:
cv_kind = 1
train_set_kind = 1

X_test1, y_test1, X_test2, y_test2 = BalanceDate.get_test_values()  
X_train, y_train, g_train = BalanceDate.get_train_values(train_set_kind)  
cvs = BalanceDate.split_cv(kind=cv_kind, train_set_kind=train_set_kind)
cvs_method = BalanceDate.split_cv(kind=cv_kind, train_set_kind=train_set_kind, return_kind=2)

print("test1")
utils.solubility_distribute(y_test1, 0, 1)
print("test2")
utils.solubility_distribute(y_test2, 0, 1)
print("train")
utils.solubility_distribute(y_train, 0, 1)

划分测试/训练集方法——1. 按照原始比例
划分测试/训练集方法——1. 按照原始比例
划分10cv
训练集数目: (5666, 1091)
划分cv方法—— 1: 默认 随机取，每种（+ - =）取原比例的
返回所有验证集
        variations decrease no-change increase
train1         567      280       193       94
train2         567      280       193       94
train3         567      280       193       94
train4         567      280       193       94
train5         567      280       193       94
train6         567      280       193       94
train7         566      280       193       93
train8         566      280       192       94
train9         566      279       193       94
train10        566      279       193       94
划分测试/训练集方法——1. 按照原始比例
划分10cv
训练集数目: (5666, 1091)
划分cv方法—— 1: 默认 随机取，每种（+ - =）取原比例的
返回使用的cv方法对象
test1
-1: 0: 1 = 12: 22: 12= 1.00: 1.83: 1.00
test2
-1: 0: 1 = 338: 237: 87= 1.00: 0.70: 0.26
train
-1: 0: 1 = 2798: 1929: 939= 1.00: 0.69: 0.34


'-1: 0: 1 = 2798: 1929: 939= 1.00: 0.69: 0.34'

# read model

In [4]:
names = ['RandomForeast', 'lightGBM', 'XGBoost']

models = [
    RandomForestClassifier(random_state=0),
    lgb.LGBMClassifier(random_state=0),  # gbm     
    xgb.XGBClassifier(random_state=0),
]

print(names)

['RandomForeast', 'lightGBM', 'XGBoost']


# train

In [5]:
res_cv = []  
res_blind = []  
for i in range(len(names)):
    name = names[i]
    model = models[i]
    print(name, model)
    print("10cv")
    stime = time.time()
    res_cv.append( utils.CVUtil(model, name, cvs_method).set_data(X_train, y_train, g_train).fit())
    etime = time.time()
    print("time:", etime - stime)
    print("blind")
    res_blind.append( utils.BlindTestUtil(model, name).set_data(X_train, y_train, X_test1, y_test1, X_test2, y_test2).fit() )

RandomForeast RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
10cv
原数据特征数目： 1081
实际数据特征数目： 1081
RandomForeast: 进行交叉验证
特征数目： 1081
train数据分布
-1    2798
 0    1929
 1     939
dtype: int64
原有数据数目：2798 : 1929 : 939
------cv 1------
cv test ：2518 : 1736 : 845
cv test ：280 : 193 : 94
------cv 2------
cv test ：2518 : 1736 : 845
cv test ：280 : 193 : 94
------cv 3------
cv test ：2518 : 1736 : 845
cv test ：280 : 193 : 94
------cv 4------
cv test ：2518 : 1736 : 845
cv test ：280 : 193 : 94
------cv 5------
cv te

# save result

In [6]:
utils.result_output(res_cv, res_blind, names, OUT_PATH, "10cv_3class", False)  
utils.result_output(res_cv, res_blind, names, OUT_PATH, "10cv_3class", True)  

输出路径为: .\out\20210525\10cv_3class_res.xlsx
输出路径为: .\out\20210525\10cv_3class_res_balanced.xlsx


(                acc                               gc2                    
       RandomForeast  lightGBM   XGBoost RandomForeast  lightGBM   XGBoost
 10cv       0.544108  0.649793  0.643307      0.137249  0.297646  0.289102
 test1      0.343434  0.202020  0.272727      0.040627  0.105189  0.024101
 test2      0.502123  0.542766  0.553031      0.112288  0.149989  0.149801,
        RandomForeast  lightGBM   XGBoost
 cv1         0.522441  0.659394  0.621501
 cv2         0.547600  0.631286  0.633473
 cv3         0.578738  0.644826  0.647274
 cv4         0.529702  0.662859  0.661198
 cv5         0.563303  0.640432  0.644397
 ...              ...       ...       ...
 cv8         0.540343  0.671562  0.640758
 cv9         0.562785  0.660555  0.668750
 cv10        0.523916  0.634775  0.639532
 test1       0.343434  0.202020  0.272727
 test2       0.502123  0.542766  0.553031
 
 [12 rows x 3 columns],
        RandomForeast  lightGBM   XGBoost
 cv1         0.104060  0.301696  0.269374
 cv2      